## Define Model

In [ ]:
from theano.sandbox import cuda

from keras.models import Model
from keras import layers
from keras.layers import Activation
from keras.layers import Dense
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.layers import Conv3D
from keras.layers import MaxPooling3D
from keras.layers import AveragePooling3D
from keras.layers import GlobalAveragePooling3D
from keras.layers import GlobalMaxPooling3D
from keras.engine.topology import get_source_inputs
from keras.utils.layer_utils import convert_all_kernels_in_model
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import _obtain_input_shape
from keras.preprocessing import image as k_image

import pandas as pd
import numpy as np

print(K.image_data_format()) # channels last

In [ ]:
# https://github.com/fchollet/deep-learning-models/blob/master/inception_v3.py
channel_axis = 4

def conv_block(x, filters, h, w, z, padding='same', strides=(1, 1, 1)):
    x = Conv3D(filters,
               (h, w, z), # height, width, z
               strides=strides,
               padding=padding,
               use_bias=False)(x)
    x = BatchNormalization(axis=channel_axis, scale=False)(x) # batch norm axis=4 ??
    x = Activation('relu')(x)
    return x

def inception_block(x):
    
    x = conv_block(x, 32, 3, 3, 3, strides=(2, 2, 2), padding='valid')
    x = conv_block(x, 32, 3, 3, 3, padding='valid') # padding same or valid ??
    x = conv_block(x, 64, 3, 3, 3)
    x = MaxPooling3D((3, 3, 3), strides=(2, 2, 2))(x)

    x = conv_block(x, 80, 1, 1, 1, padding='valid')
    x = conv_block(x, 192, 3, 3, 3, padding='valid')
    x = MaxPooling3D((3, 3, 3), strides=(2, 2, 2))(x)

    branch1 = conv_block(x, 64, 1, 1, 1)

    branch5 = conv_block(x, 48, 1, 1, 1)
    branch5 = conv_block(branch5, 64, 5, 5, 5)

    branch3 = conv_block(x, 64, 1, 1, 1)
    branch3 = conv_block(branch3, 96, 3, 3, 3)
    branch3 = conv_block(branch3, 96, 3, 3, 3)

    branch_pool = AveragePooling3D((3, 3, 3), strides=(1, 1, 1), padding='same')(x)
    branch_pool = conv_block(branch_pool, 32, 1, 1, 1)
    
    x = layers.concatenate(
            [branch1, branch5, branch3, branch_pool],
            axis=channel_axis)
    return x

In [ ]:
model_input = Input(shape=(None, None, None, 1)) # (x, y, z, channels)
x = inception_block(model_input)
x = GlobalAveragePooling3D(name='avg_pool')(x)
model_output = Dense(2, activation='softmax', name='predictions')(x)
model = Model(model_input, model_output, name='moses')
model.compile(optimizer='rmsprop', # change to Adam() or even Eve() ??
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary(line_length=125)

## Load training data

In [ ]:
labels = pd.read_csv('data/stage1_labels.csv')

# split ids into train and valid
valid_percent = 0.15
pos = labels[labels['cancer'] == 1]
neg = labels[labels['cancer'] == 0]
pos_valid_ids = set(pos.sample(frac=valid_percent)['id'])
neg_valid_ids = set(neg.sample(frac=valid_percent)['id'])
pos_train_ids = set(pos['id']).difference(pos_valid_ids)
neg_train_ids = set(neg['id']).difference(neg_valid_ids)
# len(pos_train_ids), len(neg_train_ids), len(pos_valid_ids), len(neg_valid_ids)

pos_train_ids = list(pos_train_ids)
neg_train_ids = list(neg_train_ids)
pos_valid_ids = list(pos_valid_ids)
neg_valid_ids = list(neg_valid_ids)
np.random.shuffle(pos_train_ids)
np.random.shuffle(neg_train_ids)
np.random.shuffle(pos_valid_ids)
np.random.shuffle(neg_valid_ids)

In [ ]:
def gen_data(patient_ids):
    for patient_id in iter(patient_ids):
        yield preprocess(patient_id)

## Train

In [ ]:
# training_data = np.reshape(new_image, (1,) + new_image.shape + (1,)) # where does this go ??

model.fit(training_data, np.array([[0, 1]]), batch_size=32, epochs=2)